<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/PuLP%20-%20Tutorial/Tutorial%20PuLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial PuLP

This tutorial uses the well-known Traveling-Salesperson-Problem (TSP) to explain how Python's module PuLP can be used to model and solve simple Linear Programs. 



## Problem Description
The Traveling Salesperson Problem (TSP) is one of the most famous combinatorial problems in the fields of mathematics, computer science, and operations research. The classical definition of the TSP is as follows: What is the shortest possible route for a traveling salesperson seeking to visit each city on a list exactly once and return to his city of origin? (Cook, William (2012) *In Pursuit of the Travelin Salesman*).

While the problem is easy to understand and easy to formulate as a mathematical model, solving it to optimality through complete enumeration of all feasible solutions quickly becomes intractable as the number of potential solutions grows exponentially in the number of cities to visit.

To date, there exists no algorithm that solves the TSP to optimality in polynomial time and it is widely believed that there is no such algorithm. However, heuristics are usually capable of finding very good solutions in a short computation time. 


Let's continue with an artificial small-sized data set that is used throughout this tutorial. The example consist of five cities (1 to 5) in which city 1 is the salesperson's city of origin (also referred to as depot). The following Table shows the distance between the cities. 

## Mathematical formulation
The formulation of the TSP as a mathematical model is a needed step before starting to implement and solve the problem via CPLEX. Remember that this tutorial does not provide you with an introduction to mathematical modeling and the structure of such a model is therefore only briefly described. For the purpose of the course, a graph-based formulation is used that is based on the paper from Langevin, André, Francois Soumis, and Jacques Desrosiers ("Classification of travelling salesman problem formulations". In: Operations Research Letters. 1990).

Let $N$ denote the number of nodes in the network, i.e., the number of cities. The distance for the salesperson to travel between any two nodes $i$ and $j$ is denoted with $c_{ij}$. Decision variable $x_{ij}$ is used to describe if the salesperson traverses from node $i$ to node $j$ ($x_{ij}=1$), or not ($x_{ij}=0$). Using this notation, the mathematical model for the TSP is as follows: 

Objective
$
\min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij}
\tag{1}
$

$
\sum_{i=1,\dots,N} x_{ij} = 1,~ \forall~ j = 1,\dots,N \\
\sum_{j=1,\dots,N} x_{ij} = 1,~ \forall~ i = 1,\dots,N \\
x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N \\
\text{+ subtour breaking constraints}
$

Equation \eqref{eq:tsp:objective} states the objective (distance minimization) by summing up the distances of the selected arcs. Equations \eqref{eq:tsp:inflow} and \eqref{eq:tsp:outflow} ensure that the salesperson enters and exists each node exactly once (i.e., that each node is visited exactly once), respectively. Finally, Equation \eqref{eq:tsp:domain} states the domain of the decision variables. A tour that complies with constraints \eqref{eq:tsp:inflow} to \eqref{eq:tsp:domain} can lead to so-called subtours, in which the solution comprises of \textit{several} tours rather than one tour. Figure \ref{fig:tsp_not_feasible} shows an example of a \say{solution} with subtours. Therefore, Equation \eqref{eq:tsp:subtours} generically states that further constraints are needed that break such subtours. 

\notebox{
You may wonder why the subtour breaking constraints are not immediately included in the above model. This is because several formulations that break subtours have been proposed in the literature. Three of them are considered in this tutorial and model \eqref{eq:tsp:objective} to \eqref{eq:tsp:domain} can then be used as a basis for each of those formulations.
}

In the following, a formulation from \textcite{miller1960integer}, henceforth referred to as MTZ, is used. The MTZ formulation requires one additional decision variable $u_i$ that describes the position of a node in the salesperson's tour. Equation \eqref{eq:tsp:mtz:domain} defines the domain of this variable. This variable is then used in Equation \eqref{eq:tsp:mtz:constraint} to ensure that a solution consists of exactly one tour. This is done by stating that each customer is assigned to exactly one position, that each position is assigned to exactly one customer, and that the positions correspond to the actual tour of the salesperson (represented by $x_{ij}$). With this, Equations \eqref{eq:tsp:objective} to \eqref{eq:tsp:mtz:domain} (without \eqref{eq:tsp:subtours}) constitute the mathematical model of the TSP.  
%
\begin{flalign}
    \label{eq:tsp:mtz:constraint}
    u_{i} - u_{j} + N \cdot x_{ij} \le N -1,~ \forall~ i,j = 1,\dots,N: j \ne 1 ~\text{and}~ i \ne j &&
\end{flalign}
\begin{flalign}
    \label{eq:tsp:mtz:domain}
    u_{i} \in \mathbb{Z}^{+},~ \forall~ i = 1,\dots,N &&
\end{flalign}
